In [166]:
import pandas as pd
from pymongo import MongoClient
import re
import csv


### Connection avec mongodb


In [167]:
client = MongoClient("localhost:27017")
database = client.scrapy
movies = database.movies
serie = database.serie
movies.drop()
serie.drop()

### Nettoyage des données

In [168]:
df_movie = pd.read_csv("csv/movies.csv")
df_serie = pd.read_csv("csv/serie.csv")

In [169]:
df_movie["titre_original"].replace(to_replace="Original title: ", value='', regex=True,inplace=True)


In [170]:
# df_movie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# df_serie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# def convertion_duree(s):
    
#     split_cell = re.split("\s", s)
#     duree = 0
#     for elt in split_cell:
#         if "h" in elt:
#             duree = int(re.split("h", elt)[0]) *60 +duree
#         elif "m" in elt:
#             duree =int(re.split("m", elt)[0]) +duree
#     return duree
    




# df_movie['duree'] = df_movie['duree'].apply(lambda x :  convertion_duree(str(x)))
# df_serie['duree'] = df_serie['duree'].apply(lambda x :  convertion_duree(str(x)))



In [171]:
df_movie.to_csv("csv/movies.csv",index=False)
df_serie.to_csv("csv/serie.csv",index=False)

### Import des données dans mongodb

In [172]:



header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
csvfile = open('csv/movies.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["date","duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]
        
    movies.insert_one(row)

In [173]:
header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
csvfile = open('csv/serie.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]  
        
    serie.insert_one(row)

## question n°1

In [174]:
movies.find().sort("duree",-1).limit(1)[0]

{'_id': ObjectId('6398a934c2d6119f4a22514b'),
 'acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell'],
 'date': 1939,
 'descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'duree': 238,
 'genre': ['Drama', 'Romance', 'War'],
 'pays': ['United States'],
 'public': 'Passed',
 'score': 8.2,
 'titre': 'Gone with the Wind',
 'titre_original': ''}

## Question n°2

In [175]:
five_movies=movies.find().sort("score",-1).limit(5)
for movie in five_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a2250df'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251d7'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251d4'), 'acteurs': ['Elijah Wood', 'Viggo Mortensen', 'Ian McKellen'], 'date': 2003, 'descriptions': "Gandalf and Aragorn 

## Question n°3


In [176]:
Freeman_movies = movies.find({"acteurs":{"$in":["Morgan Freeman"]}})
for movie in Freeman_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a2250df'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a22513c'), 'acteurs': ['Hilary Swank', 'Clint Eastwood', 'Morgan Freeman'], 'date': 2004, 'descriptions': 'A determined woman works with a hardened boxing trainer to become a professional.', 'duree': 132, 'genre': ['Drama', 'Sport'], 'pays': ['United States'], 'public': 'PG-13', 'score': 8.1, 'titre': 'Million Dollar Baby', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a22515f'), 'acteurs': ['Clint Eastwood', 'Gene Hackman', 'Morgan Freeman'], 'date': 1992, 'descriptions': 'Retired Old West gunslinger William Munny reluctantly takes on one l

In [177]:
Cruise_movies = movies.find({"acteurs":{"$in":["Tom Cruise"]}})
for movie in Cruise_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a225197'), 'acteurs': ['Tom Cruise', 'Jennifer Connelly', 'Miles Teller'], 'date': 2022, 'descriptions': "After thirty years, Maverick is still pushing the envelope as a top naval aviator, but must confront ghosts of his past when he leads TOP GUN's elite graduates on a mission that demands the ultimate sacrifice from those chosen to fly it.", 'duree': 130, 'genre': ['Action', 'Drama'], 'pays': ['United States'], 'public': 'PG-13', 'score': 8.4, 'titre': 'Top Gun: Maverick', 'titre_original': ''}


## Question n°4

In [178]:
horror_movies = movies.find({"genre":{"$in":["Horror"]}}).sort("score",-1).limit(3)
for movie in horror_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a2251be'), 'acteurs': ['Anthony Perkins', 'Janet Leigh', 'Vera Miles'], 'date': 1960, 'descriptions': "A Phoenix secretary embezzles $40,000 from her employer's client, goes on the run and checks into a remote motel run by a young man under the domination of his mother.", 'duree': 109, 'genre': ['Horror', 'Mystery', 'Thriller'], 'pays': ['United States'], 'public': 'R', 'score': 8.5, 'titre': 'Psycho', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251b2'), 'acteurs': ['Sigourney Weaver', 'Tom Skerritt', 'John Hurt'], 'date': 1979, 'descriptions': 'The crew of a commercial spacecraft encounter a deadly lifeform after investigating an unknown transmission.', 'duree': 117, 'genre': ['Horror', 'Sci-Fi'], 'pays': ['United Kingdom', 'United States'], 'public': 'R', 'score': 8.5, 'titre': 'Alien', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251a9'), 'acteurs': ['Jack Nicholson', 'Shelley Duvall', 'Danny Lloyd'], 'date': 1980, 'des

In [179]:
drama_movies = movies.find({"genre":{"$in":["Drama"]}}).sort("score",-1).limit(3)
for movie in drama_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a2250df'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251d7'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2251d4'), 'acteurs': ['Elijah Wood', 'Viggo Mortensen', 'Ian McKellen'], 'date': 2003, 'descriptions': "Gandalf and Aragorn 

In [180]:
comedy_movies = movies.find({"genre":{"$in":["Comedy"]}}).sort("score",-1).limit(3)
for movie in comedy_movies:
    print(movie)

{'_id': ObjectId('6398a934c2d6119f4a2251c0'), 'acteurs': ['Roberto Benigni', 'Nicoletta Braschi', 'Giorgio Cantarini'], 'date': 1997, 'descriptions': 'When an open-minded Jewish waiter and his son become victims of the Holocaust, he uses a perfect mixture of will, humor, and imagination to protect his son from the dangers around their camp.', 'duree': 116, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['Italy'], 'public': 'PG-13', 'score': 8.6, 'titre': 'Life Is Beautiful', 'titre_original': 'La vita è bella'}
{'_id': ObjectId('6398a934c2d6119f4a2251b5'), 'acteurs': ['Charles Chaplin', 'Virginia Cherrill', 'Florence Lee'], 'date': 1931, 'descriptions': '', 'duree': 87, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['United States'], 'public': 'G', 'score': 8.5, 'titre': 'City Lights', 'titre_original': ''}
{'_id': ObjectId('6398a934c2d6119f4a2250e2'), 'acteurs': ['François Cluzet', 'Omar Sy', 'Anne Le Ny'], 'date': 2011, 'descriptions': 'After he becomes a quadriplegic from a par